In [1]:
import pandas as pd

summary = pd.read_csv('data_scientist_jobs.csv')

In [2]:
summary.head()

,Job Title,Company,Qualified,JobDescription
0,"Lead Analyst, Online Business Analytics (Remote)",Home Depot / THD,N,Position Purpose: The Lead Business Analyst wi...
1,"Data Analytics Apprenticeship, September 2022 ...",Google,Y,"Applications will go live Monday May 2, 2022 P..."
2,Specialist-Business Analytics,DIRECTV,N,"At DIRECTV, we’re connecting the world through..."
3,Professional - Business Analytics - Fully remote,DIRECTV,N,"At DIRECTV our amazing people, combined with a..."
4,"Analyst, Online Analytics (Remote)",Home Depot / THD,N,"Position Purpose: The Analyst, Online Analytic..."


In [3]:
summary['Qualified'] = summary['Qualified'].replace({'Y':1, 'N':0})

In [4]:
summary.head()

,Job Title,Company,Qualified,JobDescription
0,"Lead Analyst, Online Business Analytics (Remote)",Home Depot / THD,0,Position Purpose: The Lead Business Analyst wi...
1,"Data Analytics Apprenticeship, September 2022 ...",Google,1,"Applications will go live Monday May 2, 2022 P..."
2,Specialist-Business Analytics,DIRECTV,0,"At DIRECTV, we’re connecting the world through..."
3,Professional - Business Analytics - Fully remote,DIRECTV,0,"At DIRECTV our amazing people, combined with a..."
4,"Analyst, Online Analytics (Remote)",Home Depot / THD,0,"Position Purpose: The Analyst, Online Analytic..."


In [5]:
import re
import string
import nltk

nltk.download('stopwords')

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

Stopwords = stopwords.words('english')

def Punk_Remover(s):
    s = ''.join([char for char in s if char not in frozenset(string.punctuation)])
    return s

summary['JobDescription'] = summary['JobDescription'].apply(Punk_Remover)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\jacks\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [6]:
summary['JobDescription'] = summary['JobDescription'].apply(lambda x: ' '.join([word for word in x.split() if word not in (Stopwords)]))

all_reviews = ' '.join(summary.JobDescription)
all_words = nltk.FreqDist(w.lower() for w in all_reviews.split())

word_features = list(all_words)

def document_features(document):
    document_words = set(document.split())
    features = {}
    for word in word_features:
        features['contains({})'.format(word)] = (word in document_words)
    return features


In [7]:
summary.head()

,Job Title,Company,Qualified,JobDescription
0,"Lead Analyst, Online Business Analytics (Remote)",Home Depot / THD,0,Position Purpose The Lead Business Analyst str...
1,"Data Analytics Apprenticeship, September 2022 ...",Google,1,Applications go live Monday May 2 2022 Please ...
2,Specialist-Business Analytics,DIRECTV,0,At DIRECTV we’re connecting world latest tech ...
3,Professional - Business Analytics - Fully remote,DIRECTV,0,At DIRECTV amazing people combined culture thr...
4,"Analyst, Online Analytics (Remote)",Home Depot / THD,0,Position Purpose The Analyst Online Analytics ...


In [8]:
summary['JobDescription'] = summary['JobDescription'].str.lower()

In [9]:
summary.head()

,Job Title,Company,Qualified,JobDescription
0,"Lead Analyst, Online Business Analytics (Remote)",Home Depot / THD,0,position purpose the lead business analyst str...
1,"Data Analytics Apprenticeship, September 2022 ...",Google,1,applications go live monday may 2 2022 please ...
2,Specialist-Business Analytics,DIRECTV,0,at directv we’re connecting world latest tech ...
3,Professional - Business Analytics - Fully remote,DIRECTV,0,at directv amazing people combined culture thr...
4,"Analyst, Online Analytics (Remote)",Home Depot / THD,0,position purpose the analyst online analytics ...


In [10]:
all_summary = ' '.join(summary.JobDescription)

In [11]:
all_words = nltk.FreqDist(w.lower() for w in all_summary.split())

word_features = list(all_words)

def document_features(document):
    document_words = set(document.split())
    features = {}
    for word in word_features:
        features['contains({})'.format(word)] = (word in document_words)
    return features

In [12]:
import string

all_words = [word for word in summary[['JobDescription','Qualified']] if word not in set(string.punctuation)]

In [13]:
characters_to_remove = ["''",'``','.','-',')','(','#',',']
all_words = [word for word in summary[['JobDescription','Qualified']] if word not in set(characters_to_remove)]

In [14]:
print(all_words)

['JobDescription', 'Qualified']


In [15]:
summary_lot = []

for index, row in summary[['JobDescription','Qualified']].iterrows():
        summary_lot.append(tuple(row))

In [16]:
split = 0.7

featuresets = [(document_features(d), c) for (d,c) in summary_lot]

train_set, test_set = featuresets[int(round(len(featuresets)*split,0))-1:], featuresets[:int(round(len(featuresets)*split,0))-1]

nb_classifier = nltk.NaiveBayesClassifier.train(train_set)

In [17]:
nb_classifier.show_most_informative_features(25)

Most Informative Features
contains(educationexperience) = True                1 : 0      =     19.7 : 1.0
 contains(selfdirection) = True                1 : 0      =     19.7 : 1.0
          contains(5000) = True                1 : 0      =     17.1 : 1.0
         contains(hands) = True                1 : 0      =     17.1 : 1.0
      contains(periodic) = True                1 : 0      =     17.1 : 1.0
     contains(sincerely) = True                1 : 0      =     17.1 : 1.0
     contains(announced) = True                1 : 0      =     14.5 : 1.0
       contains(breadth) = True                1 : 0      =     14.5 : 1.0
      contains(children) = True                1 : 0      =     14.5 : 1.0
    contains(conceptual) = True                1 : 0      =     14.5 : 1.0
   contains(expressions) = True                1 : 0      =     14.5 : 1.0
   contains(extensively) = True                1 : 0      =     14.5 : 1.0
      contains(hardship) = True                1 : 0      =     14.5 

In [18]:
import pandas as pd

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Flatten, LSTM, Conv1D, GlobalMaxPooling1D, Dropout, Activation
from keras.layers import Input, TextVectorization
from keras.callbacks import *
from keras.metrics import *
from keras.backend import clear_session
from keras.layers.embeddings import Embedding
from keras.initializers import Constant

import spacy

In [19]:
from sklearn.model_selection import train_test_split

X = summary['JobDescription'].astype(str)
y = summary['Qualified']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=101)

In [20]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer()

tokenizer.fit_on_texts(list(X_train))

X_train_seq  = tokenizer.texts_to_sequences(X_train) 
X_test_seq = tokenizer.texts_to_sequences(X_test)

X_train_seq  = pad_sequences(X_train_seq, maxlen=300)
X_test_seq = pad_sequences(X_test_seq, maxlen=300)

In [21]:
size_of_vocabulary=len(tokenizer.word_index) + 1
print(size_of_vocabulary)

11569


In [25]:
METRICS = [
      TruePositives(name='tp'),
      FalsePositives(name='fp'),
      TrueNegatives(name='tn'),
      FalseNegatives(name='fn'), 
      BinaryAccuracy(name='accuracy'),
      Precision(name='precision'),
      Recall(name='recall'),
      AUC(name='auc'),
      AUC(name='prc', curve='PR'),
      
]

In [23]:
clear_session()

model=Sequential()

model.add(Embedding(size_of_vocabulary,500,input_length=300,trainable=True)) 

model.add(GlobalMaxPooling1D())

model.add(Dense(64,activation='relu')) 
model.add(Dense(1,activation='sigmoid')) 

model.compile(optimizer='adam', loss='binary_crossentropy',metrics=METRICS) 

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=3)  
mc=ModelCheckpoint('best_model', monitor='val_loss', mode='min', save_best_only=True, save_weights_only=False, verbose=1)  

print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 300, 500)          5784500   
                                                                 
 global_max_pooling1d (Globa  (None, 500)              0         
 lMaxPooling1D)                                                  
                                                                 
 dense (Dense)               (None, 64)                32064     
                                                                 
 dense_1 (Dense)             (None, 1)                 65        
                                                                 
Total params: 5,816,629
Trainable params: 5,816,629
Non-trainable params: 0
_________________________________________________________________
None
